In [ ]:
#### from flask import Flask,request,jsonify,make_response
from flask_cors import CORS
import psycopg2
import os
import pandas as pd
app=Flask(__name__)
CORS(app)

@app.route("/")
def Home():
    return jsonify({"Message":"Apis Successfully Created"})


# Cookies
@app.route("/set_cokkie")
def set_Cookies():
    resp=make_response({"Message":"Login Page Successfully Created"})
    resp.set_cookie("session_id", "some-session-token")
    return resp

@app.route("/get_cokkie")
def get_Cookies():
    response=request.cookies.get("session_id")
    return response



#connectors
def connectors():
    conn=psycopg2.connect(host="localhost",port="5432",dbname="sample3",password="root",user="postgres")
    return conn



#Create table
@app.route("/create_table",methods=["GET"])
def create_table():
    if request.method == "GET":
        conn=connectors()
        cur=conn.cursor()
        cur.execute('''create table if not exists employee (emp_id serial primary key,emp_name varchar(50),skills varchar(50))''')
        data=[("Raju",["python","Reactjs"]),("Ramesh",["Reactjs","C#"])]
        data=[ (name,",".join(skills)) for name,skills in data]
        query='insert into employee  (emp_name,skills)  values (%s,%s)'
        cur.executemany(query,data)

        try:
            conn.commit()
            return jsonify({"Message":"Table Created Succesfull & Data Inserted Succesfull"})
        except Exception as e:
            conn.rollback()
            return jsonify({"Error": str(e)})
        finally:
                cur.close()
                conn.close()   
    else:
        return jsonify({"Error": "This method is not allowed"}), 405
    
    
#insert data
@app.route("/insert_data",methods=["POST"])
def insert_data():
    if request.method=="POST":
        conn=connectors()
        cur=conn.cursor()
        received_data=request.get_json()
        print("received_data--->",received_data)
        
        if isinstance(received_data,dict):
            emp_name=received_data["emp_name"]
            skills=received_data["skills"]
            data=(emp_name,skills)
            if all([emp_name,skills]):
                cur.execute("insert into employee(emp_name,skills) values (%s,%s)",data)
            else:
                return jsonify({"Error": "Missing Data"})
        elif isinstance(received_data,list):
            data=[(d["emp_name"],"".join(d["skills"])) for d in received_data]
            cur.executemany("insert into employee(emp_name,skills) values (%s,%s)",data)
        else:
            return jsonify({"error": "Data not found"})
        
        try:
            conn.commit()
            return jsonify({"message": "Successfully data inserted"})
        except Exception as e:
            conn.rollback()
            return jsonify({"error":str(e)})
        finally:
            cur.close()
            conn.close()      
    else:
        return jsonify({"Error":"This method is not allowed"}),405

#get data
@app.route("/get_data",methods=["GET"])
def get_data():
    if request.method=="GET":
        conn=connectors()
        cur=conn.cursor()
        
        limit = request.args.get('limit', default=0, type=int)
        offset=request.args.get("offset",default=0,type=int)
        
        query="select * from employee"
        params=[]
        if limit>0:
            query+=" limit %s"
            params.append(limit)
        if offset>0:
            query+=" offset %s"
            params.append(offset)
            
        cur.execute(query,tuple(params))
        table_data=cur.fetchall()
        
        columns=[desc[0] for desc in cur.description]
        json_data=[dict(zip(columns,row))    for row in table_data]
        
        try:
            conn.commit()
            return jsonify(json_data),200
        except Exception as e:
            conn.rollback()
            return jsonify({"error":str(e)}),500
        finally:
            cur.close()
            conn.close()      
    else:
        return jsonify({"Error":"This method is not allowed"}),405

    
#delete_data
@app.route("/delete_data/<int:id>",methods=["DELETE"])
def delete_data(id):
    if request.method=="DELETE":
        print("deleted id-->",id)
        conn=connectors()
        cur=conn.cursor()
        query="delete from employee where emp_id=%s"
        cur.execute(query,(id,))
        try:
            conn.commit()
            return jsonify({"Message":"Data was successfully deleted"}),200
        except Exception as e:
            conn.rollback()
            return jsonify({"error":str(e)}),500
        finally:
            cur.close()
            conn.close()      
    else:
        return jsonify({"Error":"This method is not allowed"}),405
    
@app.route("/update_data/<int:emp_id>", methods=["PUT"])  
def update_data(emp_id):
    """Update employee data based on employee ID."""
    
    if request.method=="PUT":
        conn=connectors()
        cur=conn.cursor()     
        received_data = request.get_json()
        if not received_data:
            return jsonify({"error": "No data provided"}), 400
        
        emp_name = received_data.get("emp_name")
        skills = ",".join(received_data.get("skills", []))
        query="update employee set"
        updates=[]
        params=[]
        if emp_name is not None:
            updates.append(" emp_name=%s")
            params.append(emp_name)

        if skills is not None:
            updates.append(" skills=%s")
            params.append(skills)
        
        query+=",".join(updates)+" where emp_id=%s"
        params.append(emp_id)
        
        try:
            cur.execute(query, params)
            conn.commit()
            if cur.rowcount == 0:
                return jsonify({"error": "Employee not found"}), 404
            return jsonify({"Message": "Data successfully updated"}), 200
        except Exception as e:
            conn.rollback()
            return jsonify({"Error": str(e)}), 500
        finally:
            cur.close()
            conn.close()
    else:
        return jsonify({"error": "Method not allowed. Use PUT method."}), 405
    
    
upload_file="Data"
folders={
    "csv":os.path.join(upload_file,"csv"),
    "excel":os.path.join(upload_file,"excel"),
    "json":os.path.join(upload_file,"json"),
    "pdf":os.path.join(upload_file,"pdf"),
    "xlsx":os.path.join(upload_file,"xlsx")
}

for value in folders.values():
    os.makedirs(value,exist_ok=True)

@app.route("/upload_file",methods=["POST"])
def upload_files():
    if request.method=="POST":
        conn=connectors()
        cur=conn.cursor()  
        files=request.files
        file_list=files.getlist("files")
        for file_names in file_list:
            extension_file=file_names.filename.rsplit(".",1)[1]
            upload_files=[]
            if extension_file in folders:
                file_path=os.path.join(folders[extension_file],file_names.filename)
                file_names.save(file_path)
                
            # If it's a CSV, XLSX, or JSON, process the file and return its contents
            if extension_file == 'csv':
                df = pd.read_csv(file_path)
                data = df.to_dict(orient='records')
            elif extension_file == 'xlsx':
                    df = pd.read_excel(file_path)
                    data = df.to_dict(orient='records')
            elif extension_file == 'json':
                    df = pd.read_json(file_path)
                    data = df.to_dict(orient='records')   
                    
            elif extension_file == 'txt' or extension_file == 'html':
                with open(file_path, 'r') as f:
                    data = f.read()
            elif extension_file == 'pdf':
                    data = "PDF uploaded successfully"
            upload_files.append({'filename':file_names.filename,'data':data})    
        return jsonify({"upload_files":"files"})    
    else:
        return jsonify({"error": "Method not allowed. Use PUT method."}), 405
    
if __name__=='__main__':
    app.run(host='127.0.0.1',port=9006,debug=False)

In [ ]:
from flask import Flask, request, make_response, redirect, url_for
 
app = Flask(__name__)
 
# A simple user data for demonstration purposes
users = {"deepthi@example.com": "password123"}
 
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Get the user credentials from the form
        email = request.form['email']
        password = request.form['password']
        
        # Check if the user exists and password is correct
        if email in users and users[email] == password:
            # User is authenticated, create a response
            response = make_response(redirect(url_for('dashboard')))
#             response=make_response("Login Paage successfull")
            # Set a session cookie with a unique session ID
            #httponly attribute is a security feature used in cookies that restricts access to the cookie from client-slide scripts
#             such as js.
            response.set_cookie('session_id', 'unique_session_id_12345', httponly=True, max_age=60*60*24)
            return response
        else:
            return "Invalid credentials, please try again."
    
    return '''
        <form method="post">
            Email: <input type="text" name="email"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Login">
        </form>
    '''
 
@app.route('/dashboard')
def dashboard():
    # Get the session_id from the request cookies
    session_id = request.cookies.get('session_id')
    
    if session_id == 'unique_session_id_12345':
        return "Welcome to your dashboard, Deepthi!"
    else:
        return redirect(url_for('login'))
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Sep/2024 15:44:33] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2024 15:44:33] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2024 15:44:35] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2024 15:44:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Sep/2024 15:44:45] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2024 15:45:22] "POST /login HTTP/1.1" 200 -
